# Imports

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

---

# Carga de datos

In [2]:
features = pd.read_pickle("../../features/auctions_train_ventana_1.pkl")
targets = pd.read_pickle("../../targets/targets_ventana_1_auctions.pkl")
train = features.merge(targets, left_index=True, right_index=True)

---

# Preparación del set de datos para XGBoost

In [64]:
X, y = train.iloc[:,:-1], train.iloc[:,-1]
data_dmatrix = xgb.DMatrix(data=X,label=y)
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=123)

In [65]:
xg_reg = xgb.XGBRegressor(colsample_bytree = 0.7, learning_rate = 1,
                n_estimators = 100, n_jobs=6, max_depth=7, objective="reg:linear")

In [66]:
xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.7, gamma=0, learning_rate=1, max_delta_step=0,
       max_depth=7, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=6, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [67]:
preds = xg_reg.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 93545.206149


---

# Reborn

In [36]:
X, y = train.iloc[:,:-1], train.iloc[:,-1]
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=0)

dtrain = xgb.DMatrix(data=X_train, label=y_train)
dtest = xgb.DMatrix(data=X_test, label=y_test)

param = {'max_depth':10,
         'eta':0.3,
         'silent':1,
         'objective':'reg:linear',
         'eval_metric' : 'rmse',
         'obj' : 'reg:squarederror',
         'colsample_bytree' : 0.3,
         'n_estimators' : 1,
         'min_child_weight' : 0.5,
         'random_state' : 0,
         'reg_alpha' : 0,
         'reg_lambda' : 1,
         'scale_pos_weight' : 1,
         'learning_rate' : 0.01}


num_round = 5000
stopping = 20
bst = xgb.train(param, dtrain, num_round, evals=[(dtrain, "train"), (dtest, "test")], early_stopping_rounds=stopping)
# make prediction
#preds = bst.predict(dtest)

[0]	train-rmse:157144	test-rmse:156756
Multiple eval metrics have been passed: 'test-rmse' will be used for early stopping.

Will train until test-rmse hasn't improved in 20 rounds.
[1]	train-rmse:156144	test-rmse:155763
[2]	train-rmse:155167	test-rmse:154792
[3]	train-rmse:154186	test-rmse:153817
[4]	train-rmse:153197	test-rmse:152834
[5]	train-rmse:152217	test-rmse:151861
[6]	train-rmse:151269	test-rmse:150923
[7]	train-rmse:150316	test-rmse:149976
[8]	train-rmse:149353	test-rmse:149021
[9]	train-rmse:148413	test-rmse:148088
[10]	train-rmse:147528	test-rmse:147210
[11]	train-rmse:146642	test-rmse:146332
[12]	train-rmse:145755	test-rmse:145453
[13]	train-rmse:144889	test-rmse:144595
[14]	train-rmse:143996	test-rmse:143708
[15]	train-rmse:143129	test-rmse:142847
[16]	train-rmse:142295	test-rmse:142020
[17]	train-rmse:141432	test-rmse:141165
[18]	train-rmse:140596	test-rmse:140336
[19]	train-rmse:139766	test-rmse:139513
[20]	train-rmse:138959	test-rmse:138712
[21]	train-rmse:138190	test